In [1]:
import os
import numpy as np
import nltk
from keras.models import Sequential
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, recurrent, Embedding
from keras.preprocessing.sequence import pad_sequences
import json

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def read_lines(file_path, separator, name_idx, content_idx, start_line = 0, limit = 10000):
    ln = 1
    line_added = 0
    prev_name = None
    sentences = []
    for line in open(file_path, 'r', encoding="utf-8"):

        if ln >= start_line + limit:
            if line_added % 2 == 0:
                break
            else:
                limit += 1
        elif ln < start_line:
            ln += 1
            continue
        ln += 1
        items = line.split(separator)
        name = items[name_idx].lower()
        content = items[content_idx].lower().replace('?', '').replace('!', '').replace( '.', '').replace( ',', '').replace( '-', '')
        words = content.split()
        if prev_name != name:
            sentences.append(words)
            line_added +=1
        else:
            #print(prev_name, "spoke again")
            sentences[-1].extend(words )
        prev_name = name
    #print(len(sentences))
    if len(sentences) % 2 != 0:
        sentences.pop();
    return sentences

In [3]:
#flatten_sentences = [word for sentence in sentences for word in sentence ]



In [3]:
def build_vocab_dict(sentences, padding_char, unknown_char):
    word_freq = nltk.FreqDist(np.hstack(sentences))
    word_freq.pop(padding_char, None)
    word_freq.pop(unknown_char, None)
    vocab = word_freq.most_common(VOCAB_SIZE - 2)

    vocab.insert(0, (padding_char,1))
    vocab.append( (unknown_char,1))
    
    vocab_dict = {pair[0]: id for id, pair in enumerate(vocab)}
    
    idx_dict = {idx:word for word, idx in vocab_dict.items()}
    print(idx_dict)
    return vocab_dict, idx_dict


#print(vocab_dict, idx_dict)

In [4]:
def save_vocab(vocab_dict):
    f = open("vocab_dict.json", 'w') 
    f.write(json.dumps(vocab_dict)) 
    f.close() 
def load_vocab():
    f = open("vocab_dict.json", 'r') 
    vocab_dict = json.loads(f.read() )
    f.close() 
    idx_dict = idx_dict = {idx:word for word, idx in vocab_dict.items()}
    return vocab_dict, idx_dict

In [5]:
def sentence_to_vec(sentences, vocab_dict, unknown_char, sentence_length):
    l = len(sentences)
    vec = []
    unk_idx = vocab_dict[unknown_char]

    for sen in sentences:
        vec.append( [vocab_dict[x] if x in vocab_dict else unk_idx for x in sen ][:20])

    padded = pad_sequences(vec, maxlen=sentence_length, dtype='int32')
    return padded

In [6]:
def to_one_hot(vec, sentence_length, vocab_length):
    print((len(vec), sentence_length, vocab_length))
    res = np.zeros((len(vec), sentence_length, vocab_length))
    for i, sen in enumerate(vec):
        for j, num in enumerate(sen):
            res[i, j, num] = 1
    return res

In [7]:
def create_model(x_vocab_len, x_max_len, y_vocab_len, y_max_len, hidden_size, num_layers):
    model = Sequential()

    # Creating encoder network
    model.add(Embedding(x_vocab_len, 1024, input_length=x_max_len, mask_zero=True))
    model.add(LSTM(hidden_size))
    model.add(RepeatVector(y_max_len))

    # Creating decoder network
    for _ in range(num_layers):
        model.add(LSTM(hidden_size, return_sequences=True))
    model.add(TimeDistributed(Dense(y_vocab_len)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
            optimizer='rmsprop',
            metrics=['accuracy'])
    return model


In [8]:
SENTENCE_LENGTH = 20
VOCAB_SIZE = 10000
"""
sentences = read_lines("movie_lines.txt", " +++$+++ ", 3, 4, 0, 1000)

vocab_dict, idx_dict = build_vocab_dict(sentences, ' ', 'UNK')
#save_vocab(vocab_dict)
vocab_dict, idx_dict = load_vocab()
#print(vocab_dict, idx_dict)

#sentences=["hi","i'm good","hello","hello","hey","hey there", 'shit','i know right']
#sentences= sentences + sentences + sentences + sentences
#sentences= sentences + sentences + sentences + sentences
#sentences= sentences + sentences + sentences + sentences


x_sentences = [sentences[i] for i in range(len(sentences)) if i % 2 == 0]
y_sentences = [sentences[i] for i in range(len(sentences)) if i % 2 == 1]

x_vec = sentence_to_vec(x_sentences, vocab_dict, 'UNK', SENTENCE_LENGTH)
y_vec = sentence_to_vec(y_sentences, vocab_dict, 'UNK', SENTENCE_LENGTH)
y_vec = to_one_hot(y_vec, SENTENCE_LENGTH, VOCAB_SIZE)

print(x_vec.shape, y_vec.shape)

model = create_model(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, 1024, 3)
#model.summary()
model.load_weights("new_chatbot_model.h5")
"""

'\nsentences = read_lines("movie_lines.txt", " +++$+++ ", 3, 4, 0, 1000)\n\nvocab_dict, idx_dict = build_vocab_dict(sentences, \' \', \'UNK\')\n#save_vocab(vocab_dict)\nvocab_dict, idx_dict = load_vocab()\n#print(vocab_dict, idx_dict)\n\n#sentences=["hi","i\'m good","hello","hello","hey","hey there", \'shit\',\'i know right\']\n#sentences= sentences + sentences + sentences + sentences\n#sentences= sentences + sentences + sentences + sentences\n#sentences= sentences + sentences + sentences + sentences\n\n\nx_sentences = [sentences[i] for i in range(len(sentences)) if i % 2 == 0]\ny_sentences = [sentences[i] for i in range(len(sentences)) if i % 2 == 1]\n\nx_vec = sentence_to_vec(x_sentences, vocab_dict, \'UNK\', SENTENCE_LENGTH)\ny_vec = sentence_to_vec(y_sentences, vocab_dict, \'UNK\', SENTENCE_LENGTH)\ny_vec = to_one_hot(y_vec, SENTENCE_LENGTH, VOCAB_SIZE)\n\nprint(x_vec.shape, y_vec.shape)\n\nmodel = create_model(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, 1024, 3)\n#mo

In [13]:
def load_data(start_line, limit, vocab_dict):
    sentences = read_lines("movie_lines.txt", " +++$+++ ", 3, 4, start_line, limit)

    x_sentences = [sentences[i] for i in range(len(sentences)) if i % 2 == 0]
    y_sentences = [sentences[i] for i in range(len(sentences)) if i % 2 == 1]

    x_vec = sentence_to_vec(x_sentences, vocab_dict, 'UNK', SENTENCE_LENGTH)
    y_vec = sentence_to_vec(y_sentences, vocab_dict, 'UNK', SENTENCE_LENGTH)
    y_vec = to_one_hot(y_vec, SENTENCE_LENGTH, VOCAB_SIZE)

    print(x_vec.shape, y_vec.shape)
    return x_vec, y_vec

In [11]:
model = create_model(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, 1024, 3)
#model.summary()
model.load_weights("new_chatbot_model.h5")

ResourceExhaustedError: OOM when allocating tensor of shape [1024,4096] and type float
	 [[Node: lstm_6/recurrent_kernel/initial_value = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [1024,4096] values: [0.00346009713 0.00873064063 -0.00158326747]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'lstm_6/recurrent_kernel/initial_value', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/platform

In [14]:
vocab_dict, idx_dict = load_vocab()
x_vec, y_vec = load_data(3000, 1000, vocab_dict)

(439, 20, 10000)
(439, 20) (439, 20, 10000)


In [ ]:
"""batch_size = 1000
start_point = 3000
batch = 3
round = 0
while True:
    print("Getting data:", start_point, batch_size)
    x_vec, y_vec, vocab_dict, idx_dict = load_data(start_point, batch_size)
    if len(x_vec) == 0:
        start_point = 0
        batch = 0
        x_vec, y_vec, vocab_dict, idx_dict = load_data(start_point, batch_size)
        round += 1
    model.fit(x_vec, y_vec, batch_size=200, nb_epoch=100)
    model.save_weights("new_chatbot_model.h5")
    start_point += batch_size
    batch += 1
    print("--------------------------------------------------------------------------")
    print("Round:", round, "Batch:", batch)
    print("--------------------------------------------------------------------------")
    
"""    

(439, 20, 10000)
(439, 20) (439, 20, 10000)
Epoch 1/100


/usr/local/lib/python3.5/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


439/439 [==============================] - 2s 4ms/step - loss: 5.7122 - acc: 0.4397
Epoch 2/100
439/439 [==============================] - 2s 4ms/step - loss: 3.8628 - acc: 0.4994
Epoch 3/100
439/439 [==============================] - 2s 4ms/step - loss: 2.7991 - acc: 0.5798
Epoch 4/100
439/439 [==============================] - 2s 4ms/step - loss: 2.1120 - acc: 0.6466
Epoch 5/100
439/439 [==============================] - 2s 4ms/step - loss: 1.5762 - acc: 0.7210
Epoch 6/100
439/439 [==============================] - 2s 4ms/step - loss: 1.2088 - acc: 0.7823
Epoch 7/100
439/439 [==============================] - 2s 4ms/step - loss: 0.9853 - acc: 0.8199
Epoch 8/100
439/439 [==============================] - 2s 4ms/step - loss: 0.7529 - acc: 0.8711
Epoch 9/100
439/439 [==============================] - 2s 4ms/step - loss: 0.5616 - acc: 0.9130
Epoch 10/100
439/439 [==============================] - 2s 4ms/step - loss: 0.4382 - acc: 0.9377
Epoch 11/100
439/439 [=============================

444/444 [==============================] - 2s 4ms/step - loss: 0.2484 - acc: 0.9489
Epoch 68/100
444/444 [==============================] - 2s 4ms/step - loss: 0.2450 - acc: 0.9559
Epoch 69/100
444/444 [==============================] - 2s 4ms/step - loss: 0.2446 - acc: 0.9563
Epoch 70/100
444/444 [==============================] - 2s 4ms/step - loss: 0.2416 - acc: 0.9536
Epoch 71/100
444/444 [==============================] - 2s 4ms/step - loss: 0.2366 - acc: 0.9569
Epoch 72/100
444/444 [==============================] - 2s 4ms/step - loss: 0.2517 - acc: 0.9519
Epoch 73/100
444/444 [==============================] - 2s 4ms/step - loss: 0.2628 - acc: 0.9495
Epoch 74/100
444/444 [==============================] - 2s 4ms/step - loss: 0.2374 - acc: 0.9538
Epoch 75/100
444/444 [==============================] - 2s 4ms/step - loss: 0.2357 - acc: 0.9552
Epoch 76/100
444/444 [==============================] - 2s 4ms/step - loss: 0.2422 - acc: 0.9526
Epoch 77/100
444/444 [=====================

436/436 [==============================] - 2s 4ms/step - loss: 0.1562 - acc: 0.9576
Epoch 32/100
436/436 [==============================] - 2s 4ms/step - loss: 0.1648 - acc: 0.9583
Epoch 33/100
436/436 [==============================] - 2s 4ms/step - loss: 0.1617 - acc: 0.9571
Epoch 34/100
436/436 [==============================] - 2s 4ms/step - loss: 0.1622 - acc: 0.9531
Epoch 35/100
436/436 [==============================] - 2s 4ms/step - loss: 0.1667 - acc: 0.9602
Epoch 36/100
436/436 [==============================] - 2s 4ms/step - loss: 0.1547 - acc: 0.9583
Epoch 37/100
436/436 [==============================] - 2s 4ms/step - loss: 0.1676 - acc: 0.9587
Epoch 38/100
436/436 [==============================] - 2s 4ms/step - loss: 0.1731 - acc: 0.9544
Epoch 39/100
436/436 [==============================] - 2s 4ms/step - loss: 0.1963 - acc: 0.9586
Epoch 40/100
436/436 [==============================] - 2s 4ms/step - loss: 0.1811 - acc: 0.9578
Epoch 41/100
436/436 [=====================

460/460 [==============================] - 2s 4ms/step - loss: 0.0970 - acc: 0.9685
Epoch 98/100
460/460 [==============================] - 2s 4ms/step - loss: 0.0870 - acc: 0.9738
Epoch 99/100
460/460 [==============================] - 2s 4ms/step - loss: 0.0891 - acc: 0.9734
Epoch 100/100
460/460 [==============================] - 2s 4ms/step - loss: 0.0923 - acc: 0.9736
--------------------------------------------------------------------------
Round: 0 Batch: 8
--------------------------------------------------------------------------
(435, 20, 10000)
(435, 20) (435, 20, 10000)
Epoch 1/100
435/435 [==============================] - 2s 4ms/step - loss: 5.6135 - acc: 0.4156
Epoch 2/100
435/435 [==============================] - 2s 4ms/step - loss: 4.0070 - acc: 0.5167
Epoch 3/100
435/435 [==============================] - 2s 4ms/step - loss: 2.9031 - acc: 0.5832
Epoch 4/100
435/435 [==============================] - 2s 4ms/step - loss: 2.1953 - acc: 0.6440
Epoch 5/100
435/435 [=======

442/442 [==============================] - 2s 4ms/step - loss: 0.3886 - acc: 0.9025
Epoch 62/100
442/442 [==============================] - 2s 4ms/step - loss: 0.1440 - acc: 0.9674
Epoch 63/100
442/442 [==============================] - 2s 4ms/step - loss: 0.0829 - acc: 0.9814
Epoch 64/100
442/442 [==============================] - 2s 4ms/step - loss: 0.0697 - acc: 0.9829
Epoch 65/100
442/442 [==============================] - 2s 4ms/step - loss: 0.0725 - acc: 0.9826
Epoch 66/100
442/442 [==============================] - 2s 4ms/step - loss: 0.0702 - acc: 0.9826
Epoch 67/100
442/442 [==============================] - 2s 4ms/step - loss: 0.0667 - acc: 0.9844
Epoch 68/100
442/442 [==============================] - 2s 4ms/step - loss: 0.0664 - acc: 0.9808
Epoch 69/100
442/442 [==============================] - 2s 4ms/step - loss: 0.0709 - acc: 0.9828
Epoch 70/100
442/442 [==============================] - 2s 4ms/step - loss: 0.0764 - acc: 0.9808
Epoch 71/100
442/442 [=====================

443/443 [==============================] - 2s 4ms/step - loss: 0.1199 - acc: 0.9858
Epoch 26/100
443/443 [==============================] - 2s 4ms/step - loss: 0.1071 - acc: 0.9876
Epoch 27/100
443/443 [==============================] - 2s 4ms/step - loss: 0.1053 - acc: 0.9877
Epoch 28/100
443/443 [==============================] - 2s 4ms/step - loss: 0.1046 - acc: 0.9861
Epoch 29/100
443/443 [==============================] - 2s 4ms/step - loss: 0.0967 - acc: 0.9880
Epoch 30/100
443/443 [==============================] - 2s 4ms/step - loss: 0.0968 - acc: 0.9887
Epoch 31/100
443/443 [==============================] - 2s 4ms/step - loss: 0.0939 - acc: 0.9880
Epoch 32/100
443/443 [==============================] - 2s 4ms/step - loss: 0.0948 - acc: 0.9863
Epoch 33/100
443/443 [==============================] - 2s 4ms/step - loss: 0.0913 - acc: 0.9872
Epoch 34/100
443/443 [==============================] - 2s 4ms/step - loss: 0.0832 - acc: 0.9877
Epoch 35/100
443/443 [=====================

In [56]:
sen = "what's your name"
sen = sen.lower().replace('?', '').replace('!', '').replace( '.', '')
vec = sentence_to_vec([sen], vocab_dict, 'UNK', SENTENCE_LENGTH)
#print(model.predict(vec).shape)
res = model.predict(vec)

vec_y = np.argmax(res, axis=2)
" ".join([idx_dict[x] for x in vec_y[0]])

'                      you   you     but time hardly that'

In [59]:
sen = " color"
sen = sen.lower().replace('?', '').replace('!', '').replace( '.', '')
vec = sentence_to_vec([sen], vocab_dict, 'UNK', SENTENCE_LENGTH)
#print(model.predict(vec).shape)
res = model.predict(vec)

vec_y = np.argmax(res, axis=2)
" ".join([idx_dict[x] for x in vec_y[0] if idx_dict[x]!= 'UNK'])

'                              who else thank yes to'